# Libraries

In [1]:
library(caTools)

# Read data

In [2]:
root_dir="~/Documents/Education/20170623 Udemy - Machine Learning A-Z: Hands-On Python and R in Data Science/"
work_dir="Course data/Part 2 - Regression/Section 5 - Multiple Linear Regression"
setwd(paste(root_dir,work_dir,sep=''))
df=read.csv('50_Startups.csv')
str(df)
head(df)

'data.frame':	50 obs. of  5 variables:
 $ R.D.Spend      : num  165349 162598 153442 144372 142107 ...
 $ Administration : num  136898 151378 101146 118672 91392 ...
 $ Marketing.Spend: num  471784 443899 407935 383200 366168 ...
 $ State          : Factor w/ 3 levels "California","Florida",..: 3 1 2 3 2 3 1 2 3 1 ...
 $ Profit         : num  192262 191792 191050 182902 166188 ...


R.D.Spend,Administration,Marketing.Spend,State,Profit
165349.2,136897.80,471784.1,New York,192261.8
162597.7,151377.59,443898.5,California,191792.1
153441.5,101145.55,407934.5,Florida,191050.4
144372.4,118671.85,383199.6,New York,182902.0
142107.3,91391.77,366168.4,Florida,166187.9
131876.9,99814.71,362861.4,New York,156991.1


# Categorize categorical variables

In [3]:
df$State=factor(df$State)
str(df)

'data.frame':	50 obs. of  5 variables:
 $ R.D.Spend      : num  165349 162598 153442 144372 142107 ...
 $ Administration : num  136898 151378 101146 118672 91392 ...
 $ Marketing.Spend: num  471784 443899 407935 383200 366168 ...
 $ State          : Factor w/ 3 levels "California","Florida",..: 3 1 2 3 2 3 1 2 3 1 ...
 $ Profit         : num  192262 191792 191050 182902 166188 ...


# Train/test

In [4]:
set.seed(123)
split=sample.split(df$Profit, SplitRatio=0.8)
training_set=subset(df,split)
test_set=subset(df,!split)
print(noquote(paste('Training set:',paste(dim(training_set),collapse='x'))))
print(noquote(paste('Test set:',paste(dim(test_set),collapse='x'))))

[1] Training set: 40x5
[1] Test set: 10x5


# Fit linear

In [5]:
mult_reg=lm(formula=Profit~.,training_set)
summary(mult_reg)


Call:
lm(formula = Profit ~ ., data = training_set)

Residuals:
   Min     1Q Median     3Q    Max 
-33128  -4865      5   6098  18065 

Coefficients:
                  Estimate Std. Error t value Pr(>|t|)    
(Intercept)      4.977e+04  7.516e+03   6.622 1.36e-07 ***
R.D.Spend        7.986e-01  5.604e-02  14.251 6.70e-16 ***
Administration  -2.942e-02  5.828e-02  -0.505    0.617    
Marketing.Spend  3.268e-02  2.127e-02   1.537    0.134    
StateFlorida     1.162e+02  4.048e+03   0.029    0.977    
StateNew York   -1.213e+02  3.751e+03  -0.032    0.974    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 9908 on 34 degrees of freedom
Multiple R-squared:  0.9499,	Adjusted R-squared:  0.9425 
F-statistic:   129 on 5 and 34 DF,  p-value: < 2.2e-16


# Backward elimination

In [6]:
p_limit=0.05
target='Profit'
factors=names(Filter(is.factor,df))
feat_lst=colnames(df)[colnames(df)!=target]
done=FALSE
while(!done){
    formula_str=paste(paste(target,'~'),paste(feat_lst,collapse=' + '))
    be_mult_reg=lm(formula=formula_str,training_set)
    p_vals=summary(be_mult_reg)[4]$coefficients[,4]
    p_max=max(p_vals)
    if(p_max>p_limit){
        p_max_feat=names(which.max(p_vals))
        if(p_max_feat %in% feat_lst){
            print(noquote(paste('Removing feature',p_max_feat,'with p',format(p_max,digits=3))))
            if(p_max_feat=='(Intercept)'){
                feat_lst=append(c(0),feat_lst)
            } else {    
                feat_lst=feat_lst[feat_lst != p_max_feat]
            }
        } else {
            factor_to_remove=''
            factor_pos=0
            while(factor_to_remove==''){
                factor_pos=factor_pos+1
                factor_test=factors[factor_pos]
                if(substr(p_max_feat,1,nchar(factor_test))==factor_test){
                    factor_to_remove=factor_test
                }
            }
            print(noquote(paste('Removing factor',factor_to_remove,'with p',format(p_max,digits=3))))
            feat_lst=feat_lst[feat_lst != factor_to_remove]
        }
    } else {
        done=TRUE
    }
}
summary(be_mult_reg)

[1] Removing factor State with p 0.977
[1] Removing feature Administration with p 0.609
[1] Removing feature Marketing.Spend with p 0.0713



Call:
lm(formula = formula_str, data = training_set)

Residuals:
   Min     1Q Median     3Q    Max 
-34334  -4894   -340   6752  17147 

Coefficients:
             Estimate Std. Error t value Pr(>|t|)    
(Intercept) 4.902e+04  2.748e+03   17.84   <2e-16 ***
R.D.Spend   8.563e-01  3.357e-02   25.51   <2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 9836 on 38 degrees of freedom
Multiple R-squared:  0.9448,	Adjusted R-squared:  0.9434 
F-statistic: 650.8 on 1 and 38 DF,  p-value: < 2.2e-16


# Forward selection

In [22]:
p_limit=0.05
target='Profit'
factors=names(Filter(is.factor,df))
feat_lst=c()
feat_candidates=colnames(df)[colnames(df)!=target]
done=FALSE
get_reg=function(f,target,f_base,data){
    formula_str=paste(paste(target,'~'),paste(append(f_base,f),collapse=' + '))
    return(lm(formula=formula_str,data))
}
get_marginal_p=function(reg,f_base){
    num_new_features=length(reg$coefficients)-length(f_base)-1
    return(max(tail(summary(reg)[4]$coefficients[,4],num_new_features)))
}
while(!done){
    aux_reg_lst=lapply(feat_candidates,function(f) get_reg(f,target,feat_lst,training_set))
    p_vals=unlist(lapply(aux_reg_lst,function(r) get_marginal_p(r,feat_lst)))
    p_min=min(p_vals)
    if(p_min<p_limit){
        p_min_pos=which.min(p_vals)
        print(noquote(paste('Adding feature',feat_candidates[p_min_pos],'with p',format(p_min,digits=3))))
        feat_lst=append(feat_lst,feat_candidates[p_min_pos])
        feat_candidates=feat_candidates[-p_min_pos]
        fs_mult_reg=aux_reg_lst[p_min_pos][[1]]
    } else {
        done=TRUE
    }
}
summary(fs_mult_reg)

[1] Adding feature R.D.Spend with p 1.63e-25



Call:
lm(formula = formula_str, data = data)

Residuals:
   Min     1Q Median     3Q    Max 
-34334  -4894   -340   6752  17147 

Coefficients:
             Estimate Std. Error t value Pr(>|t|)    
(Intercept) 4.902e+04  2.748e+03   17.84   <2e-16 ***
R.D.Spend   8.563e-01  3.357e-02   25.51   <2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 9836 on 38 degrees of freedom
Multiple R-squared:  0.9448,	Adjusted R-squared:  0.9434 
F-statistic: 650.8 on 1 and 38 DF,  p-value: < 2.2e-16
